<h1><center><b>IMDB SENTIMENT ANALYSIS

## DOWNLOADING DATASET

In [1]:
import opendatasets as od

In [3]:
od.download("https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

100%|█████████████████████████████████████████████████████████████████████████████| 25.7M/25.7M [00:04<00:00, 6.35MB/s]


## VISUALIZING DATASET

In [4]:
import numpy as np 
import pandas as pd

In [5]:
df=pd.read_csv('imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
df.head()

review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive

## CHECKING DATASET

In [6]:
df.shape

(50000, 2)

In [7]:
df.duplicated().sum()

418

In [8]:
df.drop_duplicates(inplace=True)

In [9]:
df.shape

(49582, 2)

In [10]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

## TEXT PREPROCESSING

#### <i>LOWERING CASE

In [12]:
df['review']=df['review'].str.lower()
df.head()

review sentiment
0  one of the other reviewers has mentioned that ...  positive
1  a wonderful little production. <br /><br />the...  positive
2  i thought this was a wonderful way to spend ti...  positive
3  basically there's a family where a little boy ...  negative
4  petter mattei's "love in the time of money" is...  positive

#### <i>REMOVE HTML TAGS

In [13]:
import re

In [14]:
def remove_html_tags(text):
    pattern=re.compile('<.*?>')
    return pattern.sub(r'',text)

In [15]:
df['review']=df['review'].apply(remove_html_tags)
df.head()

review sentiment
0  one of the other reviewers has mentioned that ...  positive
1  a wonderful little production. the filming tec...  positive
2  i thought this was a wonderful way to spend ti...  positive
3  basically there's a family where a little boy ...  negative
4  petter mattei's "love in the time of money" is...  positive

#### <i>CHECKING IF URL IS PRESENT IN THE DATAFRAME

In [23]:
def check_url(text):
    pattern=re.compile(r'https?://S+|www\.\S+')
    f=re.findall(pattern,text)
    if len(f)!=0:
        print(f)                      

In [24]:
df['review'].apply(check_url)

['www.cei.org.', 'www.cei.org/pdf/5539.pdf']
['www.invocus.net)']
['www.softfordigging.com']
['www.petitiononline.com/19784444/petition.html']
['www.comingsoon.net/films.php?id=36310']
['www.residenthazard.com)']
['www.zonadvd.com']
['www.nixflix.com']
['www.abc.net.au/chaser.']
['www.lovetrapmovie.com']
['www.thepetitionsite.com']
['www.petitiononline.com/gh1215/petition.html']
['www.johntopping.com/harvey%20perr/war%20widow/war_widow.html']
['www.mediasickness.com']
['www.imdb.com/title/tt0073891/']
['www.residenthazard.com)']
['www.imdb.com/title/tt0363163/its']
['www.nixflix.com']
['www.poffysmoviemania.com)']
['www.gutenberg.org/ebooks/18137']
['www.reel13.org)']
['www.cinemablend.com/feature.php?id=209', 'www.cinemablend.com/forum/showthread.php?s=&threadid=21699']
['www.youtube.com/watch?v=rmb4-hyet_y']
['www.dvdbeaver.com/film/dvdcompare2/kingofmasks.htmwonderful']
['www.helium.com/items/1433421-sydney-white-review']
['www.imdb.com/title/tt0962736/awards']
['www.screendaily.com

0        None
1        None
2        None
3        None
4        None
         ... 
49995    None
49996    None
49997    None
49998    None
49999    None
Name: review, Length: 49582, dtype: object

#### <i>REMOVING URL

In [25]:
def remove_url(text):
    pattern=re.compile(r'https?://S+|www\.\S+')
    return pattern.sub(r'',text)

In [26]:
df['review']=df['review'].apply(remove_url)

#### <i>REMOVING PUNCTUATION

In [27]:
import string

In [29]:
exclude=string.punctuation
def remove_punc(text):
    return text.translate(str.maketrans('','',exclude))

In [32]:
df['review']=df['review'].apply(remove_punc)
df.head()

review sentiment
0  one of the other reviewers has mentioned that ...  positive
1  a wonderful little production the filming tech...  positive
2  i thought this was a wonderful way to spend ti...  positive
3  basically theres a family where a little boy j...  negative
4  petter matteis love in the time of money is a ...  positive

#### <i>REMOVING CHAT WORDS

In [35]:
chat_words={
'AFAIK':'As Far As I Know','AFK':'Away From Keyboard','ASAP':'As Soon As Possible','ATK':'At The Keyboard','ATM':'At The Moment','BAK':'Back At Keyboard','BBL':'Be Back Later','BBS':'Be Back Soon','BFN':'Bye For Now','B4N':'Bye For Now','BRB':'Be Right Back','BRT':'Be Right There','BTW':'By The Way','B4':'Before','B4N':'Bye For Now','CU':'See You','CUL8R':'See You Later','CYA':'See You','FAQ':'Frequently Asked Questions','FC':'Fingers Crossed','FYI':'For Your Information','GAL':'Get A Life','GG':'Good Game','GN':'Good Night','GMTA':'Great Minds Think Alike','GR8':'Great',
'G9':'Genius','IC':'I See','ICQ':'I Seek you','ILU':'I Love You','IMHO':'In My Honest Opinion','IMO':'In My Opinion','IOW':'In Other Words','IRL':'In Real Life','LDR':'Long Distance Relationship','LMAO':'Laugh My Ass Off','LOL':'Laughing Out Loud','LTNS':'Long Time No See','L8R':'Later','MTE':'My Thoughts Exactly','M8':'Mate','NRN':'No Reply Necessary','OIC':'Oh I See','PITA':'Pain In The Ass','PRT':'Party','PRW':'Parents Are Watching','ROFL':'Rolling On The Floor Laughing','ROFLOL':'Rolling On The Floor Laughing Out Loud','ROTFLMAO':'Rolling On The Floor Laughing My Ass Off','SK8':'Skate','THX':'Thank You','TTYL':'Talk To You Later','U':'You','U2':'You Too',
'U4E':'Yours For Ever','WB':'Welcome Back','WTF':'What The Fuck','WTG':'Way To Go','WUF':'Where Are You From','W8':'Wait','TFW' : 'That feeling when','MFW' : 'My face when','MRW' : 'My reaction when','IFYP' : 'I feel your pain','LOL' : 'Laughing out loud','TNTL' : 'Trying not to laugh','JK' : 'Just kidding','IDC' : "I don't care",'ILY' : 'I love you','IMU' : 'I miss you','ADIH' : 'Another day in hell','ZZZ' : 'Sleeping','WYWH' : 'Wish you were here','BAE' : 'Before anyone else','FIMH' : 'Forever in my heart','BSAAW' : 'Big smile and a wink','BWL' : 'Bursting with laughter','LMAO' : 'Laughing my ass off','BFF': 'Best friends forever','CSL' : 'Can’t stop laughing'}

In [50]:
def remove_chat_words(text):
    new_text=[]
    for w in text.split():
        if w.upper() in chat_words:
            #print(w,chat_words[w.upper()].lower())
            new_text.append(chat_words[w.upper()].lower())
            
        else:
            new_text.append(w)
    return " ".join(new_text)

In [52]:
df['review']=df['review'].apply(remove_chat_words)
df.head()

review sentiment
0  one of the other reviewers has mentioned that ...  positive
1  a wonderful little production the filming tech...  positive
2  i thought this was a wonderful way to spend ti...  positive
3  basically theres a family where a little boy j...  negative
4  petter matteis love in the time of money is a ...  positive

#### <i>REMOVING STOPWORDS

In [53]:
from nltk.corpus import stopwords

In [54]:
def remove_stopwords(text):
    new_text=[]
    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append('')
        else:
            new_text.append(word)
    return " ".join(new_text)

In [56]:
df['review']=df['review'].apply(remove_stopwords)
df.head()

review sentiment
0  one    reviewers  mentioned   watching  1 oz e...  positive
1   wonderful little production  filming techniqu...  positive
2   thought    wonderful way  spend time    hot s...  positive
3  basically theres  family   little boy jake thi...  negative
4  petter matteis love   time  money   visually s...  positive

In [65]:
df.head()

review sentiment
0  one    reviewers  mentioned   watching  1 oz e...  positive
1   wonderful little production  filming techniqu...  positive
2   thought    wonderful way  spend time    hot s...  positive
3  basically theres  family   little boy jake thi...  negative
4  petter matteis love   time  money   visually s...  positive

#### <i>REMOVING SPELLING ERRORS(NOT DONE)

In [58]:
import textblob

In [60]:
df['review']=df['review'].apply(lambda x:textblob.TextBlob(x).correct().string)

KeyboardInterrupt: 

#### <i>REMOVING EMOJIS(NOT DONE)

In [62]:
!pip install emoji

     -------------------------------------- 358.9/358.9 kB 4.5 MB/s eta 0:00:00


In [63]:
import emoji

In [57]:
def find_emoji(text):
    pattern=re.compile("["
                      u"\U0001F600-\U0001F64F"
                      u"\U0001F300-\U0001F5FF"
                      u"\U0001F680-\U0001F6FF"
                      u"\U0001F1E0-\U0001F1FF"
                      u"\U00002702-\U000027B0"
                      u"\U000024C2-\U0001F251" 
                      "]+",flags=re.UNICODE)
    f=re.findall(pattern,text)
    if len(f)!=0:
        print(f)

In [64]:
df['review'].apply(find_emoji)

error: unterminated character set at position 0

In [66]:
def remove_emoji(text):
    pattern=re.compile("["
                      u"\U0001F600-\U0001F64F"
                      u"\U0001F300-\U0001F5FF"
                      u"\U0001F680-\U0001F6FF"
                      u"\U0001F1E0-\U0001F1FF"
                      u"\U00002702-\U000027B0"
                      u"\U000024C2-\U0001F251" 
                      "]+",flags=re.UNICODE)
    return pattern.sub(r'',text)

In [69]:
df['review']=df['review'].apply(remove_emoji)

#### <i>LEMMATIZATION

In [74]:
from nltk.stem import WordNetLemmatizer

In [77]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Niteesh\AppData\Roaming\nltk_data...


True

In [75]:
lemmatizer=WordNetLemmatizer()
def lem_words(text):
    new_text=[]
    for word in text.split():
        w=lemmatizer.lemmatize(word,pos='v')
        new_text.append(w) 
        
    return " ".join(new_text)

In [79]:
df['review']=df['review'].apply(lem_words)
df.head()

review sentiment
0  one reviewers mention watch 1 oz episode youll...  positive
1  wonderful little production film technique una...  positive
2  think wonderful way spend time hot summer week...  positive
3  basically theres family little boy jake think ...  negative
4  petter matteis love time money visually stun f...  positive

df2=df

df2.shape

#### <i>TOKENIZATION(NOT DONE)

In [80]:
import spacy

In [86]:
nlp=spacy.load('en_core_web_sm')
df['review']=df['review'].apply(lambda x:nlp(x))

KeyboardInterrupt: 

## WORD2VEC

In [118]:
import gensim
from gensim.utils import simple_preprocess

In [108]:
from nltk.tokenize import sent_tokenize

In [119]:
story=[]
for doc in df['review']:
    raw_sent=sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [120]:
m=gensim.models.Word2Vec(window=10,min_count=2)

In [121]:
m.build_vocab(story)

In [122]:
m.train(story,total_examples=m.corpus_count,epochs=m.epochs)

(27056888, 29341165)

In [124]:
len(m.wv.index_to_key)

69674

#### <i>AVERAGE WORD2VEC

In [93]:
from tqdm import tqdm

In [127]:
def document_vector(doc):
    doc=[word for word in doc.split() if word in m.wv.index_to_key]
    return np.mean(m.wv[doc],axis=0)

In [128]:
X=[]
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))
X=np.array(X)

100%|████████████████████████████████████████████████████████████████████████████| 49582/49582 [32:57<00:00, 25.07it/s]


## LABEL ENCODING

In [129]:
from sklearn.preprocessing import LabelEncoder

In [130]:
le=LabelEncoder()
y=le.fit_transform(df['sentiment'])

In [131]:
y

array([1, 1, 1, ..., 0, 0, 0])

## TRAIN TEST SPLIT

In [132]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import optuna

In [133]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=1)

## HYPERPARAMETER TUNING USING OPTUNA MODULE

In [136]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 210,20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_depth = trial.suggest_int('max_depth', 1, 20)
    model=RandomForestClassifier(n_estimators=n_estimators,criterion='gini',max_depth=max_depth,
                                 min_samples_split=min_samples_split,min_samples_leaf=min_samples_leaf)
    model.fit(x_train,y_train)
    y_pred=model.predict(x_test)
    return accuracy_score(y_pred,y_test)

In [137]:
study = optuna.create_study(direction='maximize')  # or 'minimize' depending on your objective
study.optimize(objective, n_trials=10)

[I 2023-08-25 23:25:11,554] A new study created in memory with name: no-name-421071c2-bd0d-4c82-ac68-ca858746418d
[I 2023-08-25 23:25:52,874] Trial 0 finished with value: 0.8183284930622782 and parameters: {'n_estimators': 190, 'min_samples_split': 4, 'min_samples_leaf': 9, 'max_depth': 6}. Best is trial 0 with value: 0.8183284930622782.
[I 2023-08-25 23:26:51,058] Trial 1 finished with value: 0.8451113262342691 and parameters: {'n_estimators': 130, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': 17}. Best is trial 1 with value: 0.8451113262342691.
[I 2023-08-25 23:26:54,262] Trial 2 finished with value: 0.755082284607938 and parameters: {'n_estimators': 70, 'min_samples_split': 7, 'min_samples_leaf': 6, 'max_depth': 1}. Best is trial 1 with value: 0.8451113262342691.
[I 2023-08-25 23:28:14,079] Trial 3 finished with value: 0.8437399161019684 and parameters: {'n_estimators': 210, 'min_samples_split': 7, 'min_samples_leaf': 10, 'max_depth': 14}. Best is trial 1 with value: 0

In [138]:
best_params = study.best_params
best_value = study.best_value
print(best_params)
print(best_value)

{'n_estimators': 130, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': 17}
0.8451113262342691


In [139]:
optuna.visualization.plot_optimization_history(study)
optuna.visualization.plot_param_importances(study)

## TRAINING

In [140]:
m=RandomForestClassifier(n_estimators=best_params['n_estimators'],
                        max_depth=best_params['max_depth'],
                        min_samples_split=best_params['min_samples_split'],
                        min_samples_leaf=best_params['min_samples_leaf'],
                        criterion='gini')
m.fit(x_train,y_train)

RandomForestClassifier(max_depth=17, min_samples_leaf=2, n_estimators=130)

## TESTING

In [141]:
y_pred=m.predict(x_test)

## EVALUATION

In [142]:
accuracy_score(y_pred,y_test)

0.8434979025492094

In [149]:
confusion_matrix(y_pred,y_test)

array([[5099,  800],
       [1140, 5357]], dtype=int64)

## SAVING MODEL

In [145]:
import pickle

In [148]:
filename = 'finalized_model.h5'
pickle.dump(m, open(filename, 'wb'))

In [150]:
# OPTIONAL
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(x_test, y_test)
print(result)

0.8434979025492094
